# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [82]:
import pandas as pd
import math as m
import nltk
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from razdel import tokenize, sentenize

In [84]:
data_default = pd.read_csv('labeled.csv')

In [85]:
data_default.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [86]:
data_default.toxic.value_counts(normalize=True)

0.0    0.66514
1.0    0.33486
Name: toxic, dtype: float64

**Обучение с дефолтной токенизацией**

In [87]:
train_def, test_def = train_test_split(data_default, test_size=0.1, shuffle=True)

In [88]:
train_def.reset_index(inplace=True)
test_def.reset_index(inplace=True)

In [89]:
vectorizer = CountVectorizer()
X_def = vectorizer.fit_transform(train_def.comment)
X_test_def = vectorizer.transform(test_def.comment)

In [90]:
X_def.shape, X_test_def.shape

((12970, 64399), (1442, 64399))

In [91]:
y_def = train_def.toxic.values
y_test_def = test_def.toxic.values

In [92]:
clf_def = LogisticRegression(C=0.1, class_weight='balanced')

In [93]:
clf_def.fit(X_def, y_def)

LogisticRegression(C=0.1, class_weight='balanced')

In [94]:
clf_def.classes_

array([0., 1.])

In [95]:
preds_def = clf_def.predict(X_test_def)

**Обучение с токенизацией razdel.tokenize**

In [98]:
def natasha_tokens(text):
    tokenized = list(tokenize(text))
    tokens = list()
    for t in tokenized:
        tokens.append(t.text)
    return tokens

In [99]:
data_razdel = pd.read_csv('labeled.csv')
data_default.toxic.value_counts(normalize=True)

0.0    0.66514
1.0    0.33486
Name: toxic, dtype: float64

In [100]:
train_rdl, test_rdl = train_test_split(data_razdel, test_size=0.1, shuffle=True)
train_rdl.reset_index(inplace=True)
test_rdl.reset_index(inplace=True) 

In [101]:
vectorizer_razdel = CountVectorizer(tokenizer = natasha_tokens)
X_rdl = vectorizer_razdel.fit_transform(train_rdl.comment)
X_test_rdl = vectorizer_razdel.transform(test_rdl.comment)

In [102]:
y_rdl = train_rdl.toxic.values
y_test_rdl = test_rdl.toxic.values

In [103]:
clf_rdl = LogisticRegression(C=0.1, class_weight='balanced')
clf_rdl.fit(X_rdl, y_rdl)

C:\Users\imphi\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=0.1, class_weight='balanced')

In [104]:
clf_rdl.classes_

array([0., 1.])

In [105]:
preds_rdl = clf_rdl.predict(X_test_rdl)

**Метрики для классификатора с дефолтной токенизацией**

In [146]:
print(classification_report(y_test_def, preds_def, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.91      0.82      0.86       962
         1.0       0.70      0.83      0.76       480

    accuracy                           0.82      1442
   macro avg       0.80      0.83      0.81      1442
weighted avg       0.84      0.82      0.83      1442



**Метрики для классификатора с токенизацией razdel.tokenize**

In [106]:
print(classification_report(y_test_rdl, preds_rdl, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.91      0.83      0.87       952
         1.0       0.72      0.85      0.78       490

    accuracy                           0.84      1442
   macro avg       0.82      0.84      0.83      1442
weighted avg       0.85      0.84      0.84      1442



Как можно видеть, абсолютно во всем показателям дефолтная токенизация проигрывает razdel.tokenize, пусть и разрыв совсем не большой: 0.01-0.03.

**Победитель - модель с токенизацией razdel.tokenize**

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [107]:
#построение таблицы

vals = [[1, 1, 1, 0, 0, 0], [1, 1, 1, 0, 0, 0], [3, 0, 1, 1, 0, 0], [1, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 1]]
words = ["я", "ты", "и", "только", "не", "он"]
docs = ['я и ты', 'ты и я', 'я, я и только я', 'только не я', 'он']
tab = pd.DataFrame(vals, columns=words, index=docs)

print(tab)

                 я  ты  и  только  не  он
я и ты           1   1  1       0   0   0
ты и я           1   1  1       0   0   0
я, я и только я  3   0  1       1   0   0
только не я      1   0  0       1   1   0
он               0   0  0       0   0   1


In [108]:
#функция нахождения параметра tf

def find_tf():
    S = list()
    for d in docs:
        s = len(d.split())
        S.append(s)
    global tf_list
    tf_list = list()
    x = 0
    for fr in vals:
        tf_sublist = [f/S[x] for f in fr]
        tf_list.append(tf_sublist)
        if x <= len(vals):
            x += 1
        else:
            x = 0    
    return tf_list

In [109]:
#функция нахождения параметра idf

def find_idf():
    N = len(docs)
    df_list = [0 for d in range(len(vals[0]))]
    for val in vals:
        for y in range(len(val)):
            if val[y] != 0:
                df_list[y] += 1
            else:
                continue
    global idf_list
    idf_list = [np.log10(N/df) for df in df_list]
    return idf_list

In [110]:
#функция для нахождения показателя tf-idf

def find_tfidf():
    global tfidf
    tfidf = list()
    for tf_sublist in tf_list:
        tfidf_sublist = list()
        for z in range(len(tf_sublist)):
            tfidf_sublist.append(tf_sublist[z]*idf_list[z])
        tfidf.append(tfidf_sublist)
    return tfidf

In [111]:
#вывод финальной таблицы

find_tf()
find_idf()
find_tfidf()

tab_new = pd.DataFrame(tfidf, columns=words, index=docs)
print(tab_new)

                        я        ты        и    только       не       он
я и ты           0.032303  0.132647  0.07395  0.000000  0.00000  0.00000
ты и я           0.032303  0.132647  0.07395  0.000000  0.00000  0.00000
я, я и только я  0.058146  0.000000  0.04437  0.079588  0.00000  0.00000
только не я      0.032303  0.000000  0.00000  0.132647  0.23299  0.00000
он               0.000000  0.000000  0.00000  0.000000  0.00000  0.69897


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к классификаторам:   
а) один должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров  
в) у классификатора должно быть задано вручную как минимум 2 параметра  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

In [144]:
#загрузка стоп-слов для использования в параметрах классификатора
from nltk.corpus import stopwords
nltk.download('stopwords')  
russian_stopwords = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\imphi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [138]:
#функция для вывода на экран топ-10 самых токсичных текстов

def show_top10(test_3, probas_3):
    sort_commentler_3b = list()
    dic_comments_3b = dict(zip(test_3.comment.tolist(), [p[1] for p in probas_3]))
    for k, v in sorted(dic_comments_3b.items(), key=lambda item: item[1]):
        sort_comment_3b = '{} : {}'.format(k, v)
        sort_commentler_3b.append(sort_comment_3b)
    j = -1
    while j > -11:
        print (sort_commentler_3b[j], '\n')
        j-=1

**1. Классификатор A: Логистическая регрессия (векторизация CountVectorizer)**

In [113]:
data_3a = pd.read_csv('labeled.csv')

In [114]:
train_3a, test_3a = train_test_split(data_3a, test_size=0.1, shuffle=True)

In [115]:
train_3a.reset_index(inplace=True)
test_3a.reset_index(inplace=True)

In [116]:
vectorizer_3a = CountVectorizer(min_df=10, max_df=0.4, lowercase = False, stop_words = russian_stopwords, ngram_range = (1,2))
X_3a = vectorizer_3a.fit_transform(train_3a.comment)
X_test_3a = vectorizer_3a.transform(test_3a.comment)

In [117]:
X_3a.shape, X_test_3a.shape

((12970, 3411), (1442, 3411))

In [118]:
y_3a = train_3a.toxic.values
y_test_3a = test_3a.toxic.values

In [119]:
clf_3a = LogisticRegression(C=0.2, class_weight='balanced')
clf_3a.fit(X_3a, y_3a)

LogisticRegression(C=0.2, class_weight='balanced')

In [120]:
clf_3a.classes_

array([0., 1.])

In [121]:
preds_3a = clf_3a.predict(X_test_3a)

In [139]:
#метрики:
print(classification_report(y_test_3a, preds_3a, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.89      0.74      0.81       959
         1.0       0.61      0.81      0.70       483

    accuracy                           0.76      1442
   macro avg       0.75      0.77      0.75      1442
weighted avg       0.79      0.76      0.77      1442



In [123]:
probas_3a = clf_3a.predict_proba(X_test_3a)

In [124]:
#десять самых токсичных текстов:
show_top10(test_3a, probas_3a)    

Раз уж вакханалия продолжается, давайте в этом треде продолжать собирать ворох банов. Копирую из прошлого треда отсюда и из ответов к нему: Ошибка: Постинг запрещён. Бан: 1067637. Причина: M Общее 6 - Злоупотребление трипкодом, аватаркой !b. Истекает: Втр Янв 22 22:00:00 2019. В новом аниме-треде ( завелся один сумасшедший модератор, который удаляет видео, а затем банит пользователей за справедливые вопросы о причине удаления данных видео по надуманным причинам вроде злоупотребления трипкодом аватаркой или обсуждения модерации вне d . Я - один из тех, кто попал под его раздачу. Поэтому, в связи с этим, требую немедленно разбанить меня и остальных, кто стал жертвой модераторского произвола и изгнать с должности модератора этого человека, он профнепригоден. Ошибка постинга: Постинг запрещён. Бан: 1068596. Причина: M Общее 6 - Злоупотребление трипкодом, аватаркой !b. Истекает: Птн Янв 25 22:00:00 2019. Какая блядь аватарка, вы в глаза долбитесь? Спасибо Абу. Ошибка постинга: Постинг запре

**2. Классификатор B: Наивный байесовский классификатор (векторизация TfidfVectorizer)**

In [125]:
data_3b = pd.read_csv('labeled.csv')

In [126]:
train_3b, test_3b = train_test_split(data_3b, test_size=0.1, shuffle=True)

In [127]:
train_3b.reset_index(inplace=True)
test_3b.reset_index(inplace=True)

In [128]:
vectorizer_3b = TfidfVectorizer(min_df=10, max_df=0.4, lowercase = False, stop_words = russian_stopwords, ngram_range = (1,2))
X_3b = vectorizer_3b.fit_transform(train_3b.comment)
X_test_3b = vectorizer_3b.transform(test_3b.comment)

In [129]:
X_3b.shape

(12970, 3386)

In [130]:
y_3b = train_3b.toxic.values
y_test_3b = test_3b.toxic.values

In [131]:
clf_3b = MultinomialNB(alpha=0.1, fit_prior = False)
clf_3b.fit(X_3b, y_3b)

MultinomialNB(alpha=0.1, fit_prior=False)

In [132]:
clf_3b.classes_

array([0., 1.])

In [133]:
preds_3b = clf_3b.predict(X_test_3b)

In [140]:
#метрики:
print(classification_report(y_test_3b, preds_3b, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.85      0.86      0.86       952
         1.0       0.73      0.71      0.72       490

    accuracy                           0.81      1442
   macro avg       0.79      0.79      0.79      1442
weighted avg       0.81      0.81      0.81      1442



In [135]:
probas_3b = clf_3b.predict_proba(X_test_3b)

In [136]:
#десять самых токсичных текстов:
show_top10(test_3b, probas_3b)    

Хохлы пидоры. У вас тоже хохол под кроватью?
 : 0.9996392603305951 

соси как хохлы сосут хохол
 : 0.9995011100824923 

НУ ВОТ, ЕРОХИН УЖЕ СВАЛИЛ ЗА БУГОР, А ТЫ ВС СИДИШЬ КАК СЫЧ В СВОЕМ ПО И СИСЯНУ ДОНАТИШЬ!!
 : 0.9994409363837877 

Какие же хохлы всадники, пиздец просто.
 : 0.9991915849255615 

НУ ЧТО, МЕДЛОХИ КАНАВСКИЕ, КАК СОСЕТСЯ, А? АЗАЗАХАХАХАХ!
 : 0.9990096621772275 

РРЯЯЯЯ, КУБАНОИДЫ НЕ РУССКИЕ, НАХУЙ ВЫ НУЖНЫ КАК ЭТО ОТДЕЛЯТЬСЯ, ЭТО ЖЕ ИСКОННО РУССКИЕ ЗЕМЛИ, ВЫХОД К ЧЕРНОМУ МОРЮ, ПЕТР ПЕРВЫЙ, РОССИЮШКА, ИМПЕРИЯ!
 : 0.9988470351128628 

Какие же хохлы новозеландцы дегенераты.
 : 0.9988248074182684 

Э че бля а? Ты че нах бл иди сюда епты.. Ты за царя гнида? Я тя щас грязь растопчу хуесос ептыбля
 : 0.9985162541304952 

Блеаадь, как же обидно когда создаешь тред, пародируешь речь ватников, случайно употребив слово из скрипта, и он скрывается у всех. Пересоздаю. Я много лет тут сижу с вами и обсераю пидорашек хоть я и сам один из них , смеюсь над их смертями, делаю фотожабы с о

**РЕЗУЛЬТАТЫ**

Результаты обучения классификаторов очень сильно различаются - ни один из текстов не попал в выборку топ-10 у обеих моделей. Тем не менее, нельзя не отметить, что все найденные моделями тексты являются очень токсичными.

У первого классификатора (логистическая регрессия) самые токсичные отличаются значительным размером - похоже, что модель выявила тексты не с максимальным процентом токсичности, а с высоким показателем токсичности в целом, а чем больше текст, тем больше количество токсичных признаков.

Второй классификатор показывает куда более короткие тексты, однако во вниманиее стоит взять и тот факт, что модели были обучены не с дефолтными настройками, а имели разные параметры, что и могло повлиять так серьёзно на результат обучения.

## *Задание 4 (2 балла)

Для классификаторов LogisticRegression, Decision Trees, Naive Bayes, Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

In [6]:
# ваш код